<a href="https://colab.research.google.com/gist/MarkCreightonQueens/3743642ed1d704e8c547c8930eb558ac/data-consolidation-negative-gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the Environment

To do: upload new minutes, rerun models with those. 

In [ ]:
#installing tools
!pip install transformers 
!pip install wget
!pip install tika


     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 3.0MB 15.1MB/s 
     |████████████████████████████████| 1.1MB 29.4MB/s 
     |████████████████████████████████| 890kB 38.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=89ce6859524b56bf746aeac3ba0cd24c1055b04c07fa8121fb22b4dfb8235f78
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=8d7697eb190b7655f10215d28eda98f3191fc61a4585ba70237607d6dd502654
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=4e0120cfda57d2cf005921a37fdbd01d40387c8bf0bd709d3c66823b288721ca
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b

In [ ]:
#Importing tools
import pandas as pd
import numpy as np
from tika import parser
#Note: need to run older version of tensor flor for BERT to work. 
%tensorflow_version 1.x
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import os
import shutil
import glob
import nltk
nltk.download('punkt')
import wget
import urllib.request
import re
from nltk.tokenize import sent_tokenize
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
import matplotlib.pyplot as plt
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
import datetime
import csv

TensorFlow 1.x selected.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#get GPU device name
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
  print('Found GPU at:{}'.format(device_name))
else:
  raise SystemError('GPU device not found')

Found GPU at:/device:GPU:0


In [ ]:
#Determining what GPU is available for use
if torch.cuda.is_available():
  device = torch.device("cuda")
  print ('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else: 
  print('No GPU available, using CPU instead.')
  device = torch.device("CPU")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


# Data Import

In [ ]:
#text data
Output = pd.read_csv('/content/drive/My Drive/Thesis/Output_backup.csv')
print(Output.shape)
#sentiment scores for multinomial naive bayes 0_1
MNB_0_1_results = pd.read_csv('/content/drive/My Drive/Thesis/Multinomial Naive Bayesian/0_1/MNB_0_1_results.csv')
print(MNB_0_1_results.shape)
#sentiment scores for multinomial naive bayes 0_4
MNB_0_4_results = pd.read_csv('/content/drive/My Drive/Thesis/Multinomial Naive Bayesian/0_4/MNB_0_4_results.csv')
print(MNB_0_4_results.shape)
#sentiment scores for logistic regression
LR_0_1_results = pd.read_csv('/content/drive/My Drive/Thesis/Logistic Regression/LR_0_1_results.csv')
print(LR_0_1_results.shape)
#sentiment scores for ordinal logistic regression
OLR_0_4_results = pd.read_csv('/content/drive/My Drive/Thesis/Ordinal Logistic Regression/OLR_0_4_results.csv')
print(OLR_0_4_results.shape)
#sentiment scores for BERT 0_1
label_results=[]
pd.DataFrame()
with open('/content/drive/My Drive/Thesis/BERT/0_1/bert_output/test_results.tsv') as file:
    rd = csv.reader(file, delimiter="\t")
    for row in rd:
      data_1=[float(i) for i in row]
      label_results.append(data_1.index(max(data_1)))
BERT_0_1_results=pd.DataFrame()
BERT_0_1_results=pd.DataFrame(label_results)
print(BERT_0_1_results.shape)
#sentiment scores for BERT 0_4
label_results=[]
pd.DataFrame()
with open('/content/drive/My Drive/Thesis/BERT/0_4/bert_output/test_results.tsv') as file:
    rd = csv.reader(file, delimiter="\t")
    for row in rd:
      data_1=[float(i) for i in row]
      label_results.append(data_1.index(max(data_1)))
BERT_0_4_results=pd.DataFrame()
BERT_0_4_results=pd.DataFrame(label_results)
print(BERT_0_4_results.shape)

(42083, 15)
(42083, 2)
(42083, 2)
(42083, 2)
(42083, 2)
(42083, 1)
(42083, 1)


In [ ]:
Output['MNB_0_1_results'] = MNB_0_1_results['0']
Output['MNB_0_1_results'] = Output['MNB_0_1_results'].replace(0,-1)
Output['MNB_0_4_results'] = MNB_0_4_results['0']
Output['MNB_0_4_results'] = Output['MNB_0_4_results'].replace(0,-2)
Output['MNB_0_4_results'] = Output['MNB_0_4_results'].replace(1,-1)
Output['MNB_0_4_results'] = Output['MNB_0_4_results'].replace(2,0)
Output['MNB_0_4_results'] = Output['MNB_0_4_results'].replace(3,1)
Output['MNB_0_4_results'] = Output['MNB_0_4_results'].replace(4,2)
Output['LR_0_1_results'] = LR_0_1_results['0']
Output['LR_0_1_results'] = Output['LR_0_1_results'].replace(0,-1)
Output['OLR_0_4_results'] = OLR_0_4_results['0'] 
Output['OLR_0_4_results'] = Output['OLR_0_4_results'].replace(0,-2)
Output['OLR_0_4_results'] = Output['OLR_0_4_results'].replace(1,-1)
Output['OLR_0_4_results'] = Output['OLR_0_4_results'].replace(2,0)
Output['OLR_0_4_results'] = Output['OLR_0_4_results'].replace(3,1)
Output['OLR_0_4_results'] = Output['OLR_0_4_results'].replace(4,2)
Output['BERT_0_1_results'] = BERT_0_1_results[0]
Output['BERT_0_1_results'] = Output['BERT_0_1_results'].replace(0,-1)
Output['BERT_0_4_results'] = BERT_0_4_results[0]
Output['BERT_0_4_results'] = Output['BERT_0_4_results'].replace(0,-2)
Output['BERT_0_4_results'] = Output['BERT_0_4_results'].replace(1,-1)
Output['BERT_0_4_results'] = Output['BERT_0_4_results'].replace(2,0)
Output['BERT_0_4_results'] = Output['BERT_0_4_results'].replace(3,1)
Output['BERT_0_4_results'] = Output['BERT_0_4_results'].replace(4,2)

In [ ]:
#rescaling 0_4 to be 0_1 fo comparison's sake
Output['MNB_0_4_results'] = Output['MNB_0_4_results'].div(2)
Output['OLR_0_4_results'] = Output['OLR_0_4_results'].div(2)
Output['BERT_0_4_results'] = Output['BERT_0_4_results'].div(2)

In [ ]:
#Calculate Average by Paragraph, then by ID
Output['Paragraph_Average_MNB_0_1'] = Output.groupby('Meta Paragraph ID')['MNB_0_1_results'].transform('mean')
temp = pd.Series(Output.groupby(["ID","Meta Paragraph ID"]).mean().groupby("ID").mean()['Paragraph_Average_MNB_0_1'],index=Output['ID'])
Output['Average_From_Paragraphs_MNB_0_1'] = temp.values
print(Output['Average_From_Paragraphs_MNB_0_1'].max())

#Calculate average by ID
Output['Average_From_Sentences_MNB_0_1'] = Output.groupby('ID')['MNB_0_1_results'].transform('mean')
print(Output['Average_From_Sentences_MNB_0_1'].max())

-0.5
-0.4329896907216495


In [ ]:
#Calculate Average by Paragraph, then by ID
Output['Paragraph_Average_MNB_0_4'] = Output.groupby('Meta Paragraph ID')['MNB_0_4_results'].transform('mean')
temp = pd.Series(Output.groupby(["ID","Meta Paragraph ID"]).mean().groupby("ID").mean()['Paragraph_Average_MNB_0_4'],index=Output['ID'])
Output['Average_From_Paragraphs_MNB_0_4'] = temp.values
print(Output['Average_From_Paragraphs_MNB_0_4'].max())

#Calculate average by ID
Output['Average_From_Sentences_MNB_0_4'] = Output.groupby('ID')['MNB_0_4_results'].transform('mean')
print(Output['Average_From_Sentences_MNB_0_4'].max())

0.6840104560342656
0.711340206185567


In [ ]:
#Calculate Average by Paragraph, then by ID
Output['Paragraph_Average_LR_0_1'] = Output.groupby('Meta Paragraph ID')['LR_0_1_results'].transform('mean')
temp = pd.Series(Output.groupby(["ID","Meta Paragraph ID"]).mean().groupby("ID").mean()['Paragraph_Average_LR_0_1'],index=Output['ID'])
Output['Average_From_Paragraphs_LR_0_1'] = temp.values
print(Output['Average_From_Paragraphs_LR_0_1'].max())

#Calculate average by ID
Output['Average_From_Sentences_LR_0_1'] = Output.groupby('ID')['LR_0_1_results'].transform('mean')
print(Output['Average_From_Sentences_LR_0_1'].max())

-0.3716082659478886
-0.39473684210526316


In [ ]:
#Calculate Average by Paragraph, then by ID
Output['Paragraph_Average_OLR_0_4'] = Output.groupby('Meta Paragraph ID')['OLR_0_4_results'].transform('mean')
temp = pd.Series(Output.groupby(["ID","Meta Paragraph ID"]).mean().groupby("ID").mean()['Paragraph_Average_OLR_0_4'],index=Output['ID'])
Output['Average_From_Paragraphs_OLR_0_4'] = temp.values
print(Output['Average_From_Paragraphs_OLR_0_4'].max())


#Calculate average by ID
Output['Average_From_Sentences_OLR_0_4'] = Output.groupby('ID')['OLR_0_4_results'].transform('mean')
print(Output['Average_From_Sentences_OLR_0_4'].max())

-0.07118226600985221
-0.12954545454545455


In [ ]:
#Calculate Average by Paragraph, then by ID
Output['Paragraph_Average_BERT_0_1'] = Output.groupby('Meta Paragraph ID')['BERT_0_1_results'].transform('mean')
temp = pd.Series(Output.groupby(["ID","Meta Paragraph ID"]).mean().groupby("ID").mean()['Paragraph_Average_BERT_0_1'],index=Output['ID'])
Output['Average_From_Paragraphs_BERT_0_1'] = temp.values
print(Output['Average_From_Paragraphs_BERT_0_1'].max())


#Calculate average by ID
Output['Average_From_Sentences_BERT_0_1'] = Output.groupby('ID')['BERT_0_1_results'].transform('mean')
print(Output['Average_From_Sentences_BERT_0_1'].max())

-0.5
-0.5


In [ ]:
#Calculate Average by Paragraph, then by ID
Output['Paragraph_Average_BERT_0_4'] = Output.groupby('Meta Paragraph ID')['BERT_0_4_results'].transform('mean')
temp = pd.Series(Output.groupby(["ID","Meta Paragraph ID"]).mean().groupby("ID").mean()['Paragraph_Average_BERT_0_4'],index=Output['ID'])
Output['Average_From_Paragraphs_BERT_0_4'] = temp.values
print(Output['Average_From_Paragraphs_BERT_0_4'].max())


#Calculate average by ID
Output['Average_From_Sentences_BERT_0_4'] = Output.groupby('ID')['BERT_0_4_results'].transform('mean')
print(Output['Average_From_Sentences_BERT_0_4'].max())

0.0
0.0


In [ ]:
os.chdir('/content/drive/My Drive/Thesis')
outfile = 'sentiment_values_sentences.csv'
Output.to_csv(outfile)

In [ ]:
os.chdir('/content')

import datetime

numeric_data = pd.read_csv('/content/drive/My Drive/Thesis/numeric_data_final.csv')
numeric_data

Date = pd.Series(numeric_data['Date'])

def add_padding_zeroes_to_date(string):
    output = []
    string = string.split('/')
    for i in string:
        if len(i) < 2:
            results = '0' + i
            output.append(results)
        else: 
            output.append(i)
    output = '/'.join(output)
    return output

Date = Date.apply(add_padding_zeroes_to_date)
numeric_data['Date'] = Date

#Taking a single observation from each FOMC meeting minute, sentiments are aggregated at this level so no information lost in doing this. 
Output_slice = Output.groupby('ID').first()
print(Output_slice.shape)

def create_date_time_object(string):
    output = datetime.datetime.strptime(string, '%m/%d/%Y')
    return output

Output_slice['output_date_time'] = Output_slice['dates'].apply(create_date_time_object)
numeric_data['numeric_date_time'] = numeric_data['Date'].apply(create_date_time_object)


(176, 38)


In [ ]:
Output_slice

,Unnamed: 0,dates_temp,dates,Paragraphs,temp,Start flag,Keep,Meta Paragraph ID,Paragraph ID,Sentences,Sentences_tokenized,Sentences_tokenized_length,Meta Sentence ID,Sentence ID,MNB_0_1_results,MNB_0_4_results,LR_0_1_results,OLR_0_4_results,BERT_0_1_results,BERT_0_4_results,Paragraph_Average_MNB_0_1,Average_From_Paragraphs_MNB_0_1,Average_From_Sentences_MNB_0_1,Paragraph_Average_MNB_0_4,Average_From_Paragraphs_MNB_0_4,Average_From_Sentences_MNB_0_4,Paragraph_Average_LR_0_1,Average_From_Paragraphs_LR_0_1,Average_From_Sentences_LR_0_1,Paragraph_Average_OLR_0_4,Average_From_Paragraphs_OLR_0_4,Average_From_Sentences_OLR_0_4,Paragraph_Average_BERT_0_1,Average_From_Paragraphs_BERT_0_1,Average_From_Sentences_BERT_0_1,Paragraph_Average_BERT_0_4,Average_From_Paragraphs_BERT_0_4,Average_From_Sentences_BERT_0_4,output_date_time
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,19930203,02/03/1993,"6/17/2020 FRB: FOMC Minutes--February 2-3, 199...",False,False,True,0,0,"6/17/2020 FRB: FOMC Minutes--February 2-3, 199...","[101, 1020, 1013, 2459, 1013, 12609, 10424, 24...",54,0,0,-1,0.5,-1,-1.0,-1,-1.0,-1.000000,-0.773451,-0.699422,0.500000,0.165669,0.221098,-1.000000,-0.724740,-0.687861,-1.000000,-0.405178,-0.339595,-1.000000,-0.858521,-0.838150,-1.000000,-0.383432,-0.276012,1993-02-03
1,208,19930323,03/23/1993,"At the start of the meeting, the subcommittee ...",True,True,True,139,0,"At the start of the meeting, the subcommittee ...","[101, 2012, 1996, 2707, 1997, 1996, 3116, 1010...",69,346,0,-1,-0.5,-1,-0.5,-1,-1.0,-1.000000,-0.672852,-0.590000,-0.166667,0.412102,0.382500,-1.000000,-0.607829,-0.580000,-0.666667,-0.357834,-0.302500,-1.000000,-0.858757,-0.880000,0.333333,-0.395320,-0.290000,1993-03-23
2,294,19930518,05/18/1993,"By unanimous vote, the minutes for the meeting...",True,True,True,198,0,"By unanimous vote, the minutes for the meeting...","[101, 2011, 13604, 3789, 1010, 1996, 2781, 200...",28,546,0,-1,0.0,-1,-0.5,-1,0.5,-1.000000,-0.771612,-0.777778,0.000000,0.313519,0.293210,-1.000000,-0.720788,-0.679012,-0.500000,-0.377175,-0.314815,-1.000000,-0.888645,-0.864198,0.500000,-0.393899,-0.256173,1993-05-18
3,377,19930707,07/07/1993,"By unanimous vote, the minutes for the meeting...",True,True,True,251,0,"By unanimous vote, the minutes for the meeting...","[101, 2011, 13604, 3789, 1010, 1996, 2781, 200...",27,708,0,-1,0.0,-1,-0.5,-1,0.5,-1.000000,-0.597192,-0.552846,0.000000,0.479121,0.451220,-1.000000,-0.710379,-0.650407,-0.500000,-0.377396,-0.296748,-1.000000,-0.889866,-0.886179,0.500000,-0.325198,-0.225610,1993-07-07
4,478,19930817,08/17/1993,"By unanimous vote, the minutes for the meeting...",True,True,True,330,0,"By unanimous vote, the minutes for the meeting...","[101, 2011, 13604, 3789, 1010, 1996, 2781, 200...",29,954,0,-1,0.0,-1,-0.5,-1,0.5,-1.000000,-0.741758,-0.684211,0.000000,0.308940,0.318713,-1.000000,-0.646795,-0.602339,-0.500000,-0.384077,-0.292398,-1.000000,-0.876511,-0.824561,0.500000,-0.332349,-0.236842,1993-08-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,17643,20140618,06/18/2014,Developments in Financial Markets and the Fede...,True,True,True,12396,0,Developments in Financial Markets and the Fede...,"[101, 8973, 1999, 3361, 6089, 1998, 1996, 2976...",63,40202,0,-1,0.5,-1,1.0,1,1.0,-1.000000,-0.719766,-0.762295,0.000000,0.272945,0.216189,-1.000000,-0.757395,-0.741803,-0.500000,-0.409690,-0.438525,-0.200000,-0.836657,-0.827869,0.300000,-0.228318,-0.289959,2014-06-18
172,17852,20140730,07/30/2014,Developments in Financial Markets and the Fede...,True,True,True,12539,0,Developments in Financial Markets and the Fede...,"[101, 8973, 1999, 3361, 6089, 1998, 1996, 2976...",62,40690,0,-1,0.5,-1,1.0,1,1.0,-1.000000,-0.799462,-0.802469,0.600000,0.221179,0.240741,-1.000000,-0.614542,-0.621399,-0.400000,-0.294153,-0.255144,-0.600000,-0.692256,-0.695473,0.400000,-0.174059,-0.141975,2014-07-30
173,17959,2014091

In [ ]:
#FOMC Meeting Minutes Summary Statistics
Output_slice
sentiment_values = Output_slice[['dates',
                                 'Average_From_Paragraphs_MNB_0_1','Average_From_Sentences_MNB_0_1',
                                 'Average_From_Paragraphs_MNB_0_4','Average_From_Sentences_MNB_0_4',
                                 'Average_From_Paragraphs_LR_0_1','Average_From_Sentences_LR_0_1',
                                 'Average_From_Paragraphs_OLR_0_4','Average_From_Sentences_OLR_0_4',
                                 'Average_From_Paragraphs_BERT_0_1','Average_From_Sentences_BERT_0_1',
                                 'Average_From_Paragraphs_BERT_0_4','Average_From_Sentences_BERT_0_4']].copy()
os.chdir('/content/drive/My Drive/Thesis')
outfile = 'sentiment_values.csv'
sentiment_values.to_csv(outfile)

In [ ]:
#Get the complete list of FOMC meetings before the given observation in numeric_data
temp = []
for i in numeric_data['numeric_date_time']:
    output = []
    for j in Output_slice['output_date_time']:
        if i > j:
            output.append(j)
    temp.append(output)
temp
#From the complete list, select the most recent 8 meetings, can be extended arbitrarily. 
Meetings = []
for i in temp:
    results = i[-8:]
    results = reversed(results)
    Meetings.append(results)
#transfer over sentiment values to numeric data.

Meetings

Meetings_df = pd.DataFrame(data = Meetings)
Output_slice_temp = Output_slice.set_index('output_date_time')
Meetings_df

,0,1,2,3,4,5,6,7
0,1993-12-21,1993-11-16,1993-09-21,1993-08-17,1993-07-07,1993-05-18,1993-03-23,1993-02-03
1,1993-12-21,1993-11-16,1993-09-21,1993-08-17,1993-07-07,1993-05-18,1993-03-23,1993-02-03
2,1994-02-04,1993-12-21,1993-11-16,1993-09-21,1993-08-17,1993-07-07,1993-05-18,1993-03-23
3,1994-03-22,1994-02-04,1993-12-21,1993-11-16,1993-09-21,1993-08-17,1993-07-07,1993-05-18
4,1994-03-22,1994-02-04,1993-12-21,1993-11-16,1993-09-21,1993-08-17,1993-07-07,1993-05-18
...,...,...,...,...,...,...,...,...
311,2014-12-17,2014-10-29,2014-09-17,2014-07-30,2014-06-18,2014-04-30,2014-03-19,2014-01-29
312,2014-12-17,2014-10-29,2014-09-17,2014-07-30,2014-06-18,2014-04-30,2014-03-19,2014-01-29
313,2014-12-17,2014-10-29,2014-09-17,2014-07-30,2014-06-18,2014-04-30,2014-03-19,2014-01-29
314,2014-12-17,2014-10-29,2014-09-17,2014-07-30,2014-06-18,2014-04-30,2014-03-19,2014-01-29


In [ ]:
temp_0 = pd.Series(Meetings_df[0])
temp_1 = pd.Series(Meetings_df[1])
temp_2 = pd.Series(Meetings_df[2])
temp_3 = pd.Series(Meetings_df[3])
temp_4 = pd.Series(Meetings_df[4])
temp_5 = pd.Series(Meetings_df[5])
temp_6 = pd.Series(Meetings_df[6])
temp_7 = pd.Series(Meetings_df[7])

temp_0 = temp_0.dropna()
temp_1 = temp_1.dropna()
temp_2 = temp_2.dropna()
temp_3 = temp_3.dropna()
temp_4 = temp_4.dropna()
temp_5 = temp_5.dropna()
temp_6 = temp_6.dropna()
temp_7 = temp_7.dropna()

numeric_data['date_0'] = temp_0
numeric_data['date_1'] = temp_1
numeric_data['date_2'] = temp_2
numeric_data['date_3'] = temp_3
numeric_data['date_4'] = temp_4
numeric_data['date_5'] = temp_5
numeric_data['date_6'] = temp_6
numeric_data['date_7'] = temp_7


In [ ]:
date_format = "%m/%d/%Y"

numeric_data['diff_0'] = numeric_data['Date'].apply(lambda x: datetime.datetime.strptime(x,date_format)) - temp_0.apply(lambda x: x)
numeric_data['diff_1'] = numeric_data['Date'].apply(lambda x: datetime.datetime.strptime(x,date_format)) - temp_1.apply(lambda x: x)
numeric_data['diff_2'] = numeric_data['Date'].apply(lambda x: datetime.datetime.strptime(x,date_format)) - temp_2.apply(lambda x: x)
numeric_data['diff_3'] = numeric_data['Date'].apply(lambda x: datetime.datetime.strptime(x,date_format)) - temp_3.apply(lambda x: x)
numeric_data['diff_4'] = numeric_data['Date'].apply(lambda x: datetime.datetime.strptime(x,date_format)) - temp_4.apply(lambda x: x)
numeric_data['diff_5'] = numeric_data['Date'].apply(lambda x: datetime.datetime.strptime(x,date_format)) - temp_5.apply(lambda x: x)
numeric_data['diff_6'] = numeric_data['Date'].apply(lambda x: datetime.datetime.strptime(x,date_format)) - temp_6.apply(lambda x: x)
numeric_data['diff_7'] = numeric_data['Date'].apply(lambda x: datetime.datetime.strptime(x,date_format)) - temp_7.apply(lambda x: x)

In [ ]:
#Output['MNB_0_1_results'] = MNB_0_1_results['0']
#Output['MNB_0_4_results'] = MNB_0_4_results['0']
#Output['LR_0_1_results'] = LR_0_1_results['0']
#Output['OLR_0_4_results'] = OLR_0_4_results['0'] 


In [ ]:
temp_0 = pd.Series(Meetings_df[0])
temp_1 = pd.Series(Meetings_df[1])
temp_2 = pd.Series(Meetings_df[2])
temp_3 = pd.Series(Meetings_df[3])
temp_4 = pd.Series(Meetings_df[4])
temp_5 = pd.Series(Meetings_df[5])
temp_6 = pd.Series(Meetings_df[6])
temp_7 = pd.Series(Meetings_df[7])

temp_0 = temp_0.dropna()
temp_1 = temp_1.dropna()
temp_2 = temp_2.dropna()
temp_3 = temp_3.dropna()
temp_4 = temp_4.dropna()
temp_5 = temp_5.dropna()
temp_6 = temp_6.dropna()
temp_7 = temp_7.dropna()

In [ ]:
temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_1'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_1'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_1'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_1'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_1'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_1'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_1'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_1'])

numeric_data['Sentiment_0_Paragraphs_MNB_0_1'] = temp_0_data
numeric_data['Sentiment_1_Paragraphs_MNB_0_1'] = temp_1_data
numeric_data['Sentiment_2_Paragraphs_MNB_0_1'] = temp_2_data
numeric_data['Sentiment_3_Paragraphs_MNB_0_1'] = temp_3_data
numeric_data['Sentiment_4_Paragraphs_MNB_0_1'] = temp_4_data
numeric_data['Sentiment_5_Paragraphs_MNB_0_1'] = temp_5_data
numeric_data['Sentiment_6_Paragraphs_MNB_0_1'] = temp_6_data
numeric_data['Sentiment_7_Paragraphs_MNB_0_1'] = temp_7_data

temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_1'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_1'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_1'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_1'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_1'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_1'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_1'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_1'])

numeric_data['Sentiment_0_Sentences_MNB_0_1'] = temp_0_data
numeric_data['Sentiment_1_Sentences_MNB_0_1'] = temp_1_data
numeric_data['Sentiment_2_Sentences_MNB_0_1'] = temp_2_data
numeric_data['Sentiment_3_Sentences_MNB_0_1'] = temp_3_data
numeric_data['Sentiment_4_Sentences_MNB_0_1'] = temp_4_data
numeric_data['Sentiment_5_Sentences_MNB_0_1'] = temp_5_data
numeric_data['Sentiment_6_Sentences_MNB_0_1'] = temp_6_data
numeric_data['Sentiment_7_Sentences_MNB_0_1'] = temp_7_data

In [ ]:
temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_4'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_4'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_4'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_4'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_4'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_4'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_4'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_MNB_0_4'])

numeric_data['Sentiment_0_Paragraphs_MNB_0_4'] = temp_0_data
numeric_data['Sentiment_1_Paragraphs_MNB_0_4'] = temp_1_data
numeric_data['Sentiment_2_Paragraphs_MNB_0_4'] = temp_2_data
numeric_data['Sentiment_3_Paragraphs_MNB_0_4'] = temp_3_data
numeric_data['Sentiment_4_Paragraphs_MNB_0_4'] = temp_4_data
numeric_data['Sentiment_5_Paragraphs_MNB_0_4'] = temp_5_data
numeric_data['Sentiment_6_Paragraphs_MNB_0_4'] = temp_6_data
numeric_data['Sentiment_7_Paragraphs_MNB_0_4'] = temp_7_data

temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_4'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_4'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_4'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_4'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_4'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_4'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_4'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_MNB_0_4'])

numeric_data['Sentiment_0_Sentences_MNB_0_4'] = temp_0_data
numeric_data['Sentiment_1_Sentences_MNB_0_4'] = temp_1_data
numeric_data['Sentiment_2_Sentences_MNB_0_4'] = temp_2_data
numeric_data['Sentiment_3_Sentences_MNB_0_4'] = temp_3_data
numeric_data['Sentiment_4_Sentences_MNB_0_4'] = temp_4_data
numeric_data['Sentiment_5_Sentences_MNB_0_4'] = temp_5_data
numeric_data['Sentiment_6_Sentences_MNB_0_4'] = temp_6_data
numeric_data['Sentiment_7_Sentences_MNB_0_4'] = temp_7_data

In [ ]:
temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_LR_0_1'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_LR_0_1'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_LR_0_1'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_LR_0_1'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_LR_0_1'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_LR_0_1'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_LR_0_1'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_LR_0_1'])

numeric_data['Sentiment_0_Paragraphs_LR_0_1'] = temp_0_data
numeric_data['Sentiment_1_Paragraphs_LR_0_1'] = temp_1_data
numeric_data['Sentiment_2_Paragraphs_LR_0_1'] = temp_2_data
numeric_data['Sentiment_3_Paragraphs_LR_0_1'] = temp_3_data
numeric_data['Sentiment_4_Paragraphs_LR_0_1'] = temp_4_data
numeric_data['Sentiment_5_Paragraphs_LR_0_1'] = temp_5_data
numeric_data['Sentiment_6_Paragraphs_LR_0_1'] = temp_6_data
numeric_data['Sentiment_7_Paragraphs_LR_0_1'] = temp_7_data

temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_LR_0_1'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_LR_0_1'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_LR_0_1'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_LR_0_1'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_LR_0_1'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_LR_0_1'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_LR_0_1'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_LR_0_1'])

numeric_data['Sentiment_0_Sentences_LR_0_1'] = temp_0_data
numeric_data['Sentiment_1_Sentences_LR_0_1'] = temp_1_data
numeric_data['Sentiment_2_Sentences_LR_0_1'] = temp_2_data
numeric_data['Sentiment_3_Sentences_LR_0_1'] = temp_3_data
numeric_data['Sentiment_4_Sentences_LR_0_1'] = temp_4_data
numeric_data['Sentiment_5_Sentences_LR_0_1'] = temp_5_data
numeric_data['Sentiment_6_Sentences_LR_0_1'] = temp_6_data
numeric_data['Sentiment_7_Sentences_LR_0_1'] = temp_7_data

In [ ]:
temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_OLR_0_4'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_OLR_0_4'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_OLR_0_4'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_OLR_0_4'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_OLR_0_4'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_OLR_0_4'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_OLR_0_4'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_OLR_0_4'])

numeric_data['Sentiment_0_Paragraphs_OLR_0_4'] = temp_0_data
numeric_data['Sentiment_1_Paragraphs_OLR_0_4'] = temp_1_data
numeric_data['Sentiment_2_Paragraphs_OLR_0_4'] = temp_2_data
numeric_data['Sentiment_3_Paragraphs_OLR_0_4'] = temp_3_data
numeric_data['Sentiment_4_Paragraphs_OLR_0_4'] = temp_4_data
numeric_data['Sentiment_5_Paragraphs_OLR_0_4'] = temp_5_data
numeric_data['Sentiment_6_Paragraphs_OLR_0_4'] = temp_6_data
numeric_data['Sentiment_7_Paragraphs_OLR_0_4'] = temp_7_data

temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_OLR_0_4'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_OLR_0_4'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_OLR_0_4'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_OLR_0_4'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_OLR_0_4'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_OLR_0_4'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_OLR_0_4'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_OLR_0_4'])

numeric_data['Sentiment_0_Sentences_OLR_0_4'] = temp_0_data
numeric_data['Sentiment_1_Sentences_OLR_0_4'] = temp_1_data
numeric_data['Sentiment_2_Sentences_OLR_0_4'] = temp_2_data
numeric_data['Sentiment_3_Sentences_OLR_0_4'] = temp_3_data
numeric_data['Sentiment_4_Sentences_OLR_0_4'] = temp_4_data
numeric_data['Sentiment_5_Sentences_OLR_0_4'] = temp_5_data
numeric_data['Sentiment_6_Sentences_OLR_0_4'] = temp_6_data
numeric_data['Sentiment_7_Sentences_OLR_0_4'] = temp_7_data

In [ ]:
temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_1'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_1'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_1'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_1'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_1'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_1'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_1'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_1'])

numeric_data['Sentiment_0_Paragraphs_BERT_0_1'] = temp_0_data
numeric_data['Sentiment_1_Paragraphs_BERT_0_1'] = temp_1_data
numeric_data['Sentiment_2_Paragraphs_BERT_0_1'] = temp_2_data
numeric_data['Sentiment_3_Paragraphs_BERT_0_1'] = temp_3_data
numeric_data['Sentiment_4_Paragraphs_BERT_0_1'] = temp_4_data
numeric_data['Sentiment_5_Paragraphs_BERT_0_1'] = temp_5_data
numeric_data['Sentiment_6_Paragraphs_BERT_0_1'] = temp_6_data
numeric_data['Sentiment_7_Paragraphs_BERT_0_1'] = temp_7_data

temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_1'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_1'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_1'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_1'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_1'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_1'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_1'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_1'])

numeric_data['Sentiment_0_Sentences_BERT_0_1'] = temp_0_data
numeric_data['Sentiment_1_Sentences_BERT_0_1'] = temp_1_data
numeric_data['Sentiment_2_Sentences_BERT_0_1'] = temp_2_data
numeric_data['Sentiment_3_Sentences_BERT_0_1'] = temp_3_data
numeric_data['Sentiment_4_Sentences_BERT_0_1'] = temp_4_data
numeric_data['Sentiment_5_Sentences_BERT_0_1'] = temp_5_data
numeric_data['Sentiment_6_Sentences_BERT_0_1'] = temp_6_data
numeric_data['Sentiment_7_Sentences_BERT_0_1'] = temp_7_data

In [ ]:
temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_4'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_4'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_4'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_4'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_4'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_4'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_4'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Paragraphs_BERT_0_4'])

numeric_data['Sentiment_0_Paragraphs_BERT_0_4'] = temp_0_data
numeric_data['Sentiment_1_Paragraphs_BERT_0_4'] = temp_1_data
numeric_data['Sentiment_2_Paragraphs_BERT_0_4'] = temp_2_data
numeric_data['Sentiment_3_Paragraphs_BERT_0_4'] = temp_3_data
numeric_data['Sentiment_4_Paragraphs_BERT_0_4'] = temp_4_data
numeric_data['Sentiment_5_Paragraphs_BERT_0_4'] = temp_5_data
numeric_data['Sentiment_6_Paragraphs_BERT_0_4'] = temp_6_data
numeric_data['Sentiment_7_Paragraphs_BERT_0_4'] = temp_7_data

temp_0_data = temp_0.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_4'])
temp_1_data = temp_1.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_4'])
temp_2_data = temp_2.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_4'])
temp_3_data = temp_3.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_4'])
temp_4_data = temp_4.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_4'])
temp_5_data = temp_5.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_4'])
temp_6_data = temp_6.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_4'])
temp_7_data = temp_7.apply(lambda x: Output_slice_temp.loc[x,'Average_From_Sentences_BERT_0_4'])

numeric_data['Sentiment_0_Sentences_BERT_0_4'] = temp_0_data
numeric_data['Sentiment_1_Sentences_BERT_0_4'] = temp_1_data
numeric_data['Sentiment_2_Sentences_BERT_0_4'] = temp_2_data
numeric_data['Sentiment_3_Sentences_BERT_0_4'] = temp_3_data
numeric_data['Sentiment_4_Sentences_BERT_0_4'] = temp_4_data
numeric_data['Sentiment_5_Sentences_BERT_0_4'] = temp_5_data
numeric_data['Sentiment_6_Sentences_BERT_0_4'] = temp_6_data
numeric_data['Sentiment_7_Sentences_BERT_0_4'] = temp_7_data

In [ ]:
print(numeric_data.shape)
os.chdir('/content/drive/My Drive/Thesis')
final_data = numeric_data
outfile = 'final_data_negative_gradient.csv'
final_data.to_csv(outfile)

(316, 176)
